In [1]:
# Import necessary libraries and modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset


In [2]:

dataset = load_dataset("deepset/prompt-injections")
# Access train and test splits
data_train = dataset['train']
data_test = dataset['test']

README.md:   0%|          | 0.00/500 [00:00<?, ?B/s]

(…)-00000-of-00001-9564e8b05b4757ab.parquet:   0%|          | 0.00/40.3k [00:00<?, ?B/s]

(…)-00000-of-00001-701d16158af87368.parquet:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/546 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/116 [00:00<?, ? examples/s]

In [3]:
data_test = data_test.to_pandas()

data_test.rename(columns={"text": "prompt"}, inplace=True)

In [4]:
# Import pipeline functionality from the Hugging Face transformer's library
from transformers import pipeline
# Load the zero-shot classification pipeline with RoBERTa
classifier = pipeline(task="zero-shot-classification", model="xlm-roberta-large")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [5]:
# Define a function to classify the prompt
def classify_prompt(prompt):
    
    # List of candidate labels (in this case, indicating whether the text is an injection or not)
    candidate_labels = ["Injection", "Normal"]
    
    # Perform zero-shot classification
    output = classifier(prompt, candidate_labels)
    
    # Return the results
    return 1 if output['labels'][0] == "Injection" else 0
# Apply classifier to the testing dataset
data_test["predicted_label"] = data_test["prompt"].apply(classify_prompt)

In [6]:
# Import performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Initialize a DataFrame to keep track of the models' performance
results = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1 score"])

# Initialize actual and predicted labels
y_test = data_test["label"]
y_predict = data_test["predicted_label"]
    
# Calculate performance metrics
accuracy = accuracy_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)

# Store performance metrics
results.loc["Testing Data"] = [accuracy, precision, recall, f1]
# Check obtained results
results

,accuracy,precision,recall,f1 score
Testing Data,0.474138,0.49505,0.833333,0.621118
